In [1]:
from gammapy.datasets import MapDataset, Dataset, Datasets
from gammapy.modeling.models import SpectralModel,PowerLawSpectralModel, Model, Models, SkyModel, GaussianSpatialModel
from gammapy.modeling import Parameter, Parameters, PriorParameter
import numpy as np
from  matplotlib import pyplot as plt
from gammapy.modeling.models import PriorModel, GaussianPriorModel, MultiDimensionalPrior, UniformPriorModel

In [2]:
covariance_matrix = np.array([[1.0, -1.0], [-1.0, 1.0]])

spatial_model = GaussianSpatialModel(
    lon_0="0.2 deg", lat_0="0.1 deg", sigma="0.3 deg", frame="galactic"
)

pwl1 = PowerLawSpectralModel(index = 2.0)
pwl2 = PowerLawSpectralModel(index = 2.1)


models  = Models([SkyModel(pwl1, spatial_model=spatial_model, name = '1'), 
                  SkyModel(pwl2, spatial_model=spatial_model, name = '2')])


uni =  UniformPriorModel(uni  = 10, modelparameters= pwl1.parameters['amplitude'])

pwl1.parameters['amplitude']

Parameter(name='amplitude', value=1e-12, factor=1e-12, scale=1.0, unit=Unit("1 / (TeV s cm2)"), min=nan, max=nan, frozen=False, prior=<gammapy.modeling.models.prior.UniformPriorModel object at 0x7efda2c17dc0>, id=0x7efda2c12100)

In [3]:
modelparameters = Parameters([pwl1.index,  pwl2.index])
multidimmodel = MultiDimensionalPrior(modelparameters,
                                          covariance_matrix = covariance_matrix,
                                          name = 'indices correlation')

   
print(multidimmodel.to_dict(full_output=True))

print(uni())
print(multidimmodel())

{'prior': {'type': 'MultiDimensionalPrior', 'modelparameters': ['index', 'index'], 'weight': 1, 'name': 'indices correlation', 'covariance_matrix': array([[ 1., -1.],
       [-1.,  1.]])}}
10.0
0.010000000000000009


In [4]:
priors = [p.prior for p in models.parameters if p.prior is not None]
print(priors)

[<gammapy.modeling.models.prior.MultiDimensionalPrior object at 0x7efda0389400>, <gammapy.modeling.models.prior.UniformPriorModel object at 0x7efda2c17dc0>, <gammapy.modeling.models.prior.MultiDimensionalPrior object at 0x7efda0389400>]


In [5]:
from gammapy.stats import prior_fit_statistic

In [6]:
prior_fit_statistic(priors)


10.01

In [7]:
mapdataset = MapDataset.read('/home/katrin/Documents/simulated_dataset.fits')
mapdataset2 = MapDataset.read('/home/katrin/Documents/simulated_dataset.fits', name = "test")

In [8]:
mapdataset.models = models

In [9]:
mapdataset.stat_sum()

in map dataset: prior_fit_statistic 10.01


568047.7444260732

In [10]:
mapdataset.stat_sum()

in map dataset: prior_fit_statistic 10.01


568047.7444260732

In [11]:
mapdataset.models.parameters['index'].value  = 10
mapdataset.stat_sum()

in map dataset: prior_fit_statistic 72.41


25164106537.8808

### Problem: Datasets

In [12]:
datasets = Datasets([mapdataset, mapdataset2, mapdataset2.copy()])

In [13]:
mapdataset

in map dataset: prior_fit_statistic 72.41


In [14]:
mapdataset2

In [15]:
datasets[0].priors

In [16]:
datasets[1].models = models
datasets[1].priors

In [17]:
datasets.models

In [18]:
priors = {}
prior_fit_statistic(list(priors.values()))

0.0

In [19]:

prior_fit_statistic(list(datasets.priors.values()))

72.41

In [20]:
datasets[2].models = datasets[1].models
datasets[2].models = None 

print(datasets[2].priors)
print(datasets[2].models)

None
None


In [21]:
datasets[2].stat_sum()

in map dataset: prior_fit_statistic 0.0


569521.2946201434

In [22]:
datasets.stat_sum()

 prior_fit_statistic(list(self.priors.values())) 72.41
in map dataset: prior_fit_statistic 72.41
in map dataset: prior_fit_statistic 72.41
in map dataset: prior_fit_statistic 0.0


50328782524.64622

In [23]:
datasets[1].stat_sum()

in map dataset: prior_fit_statistic 72.41


25164106537.8808

In [24]:
datasets.stat_sum() 

 prior_fit_statistic(list(self.priors.values())) 72.41
in map dataset: prior_fit_statistic 72.41
in map dataset: prior_fit_statistic 72.41
in map dataset: prior_fit_statistic 0.0


50328782524.64622

In [25]:
datasets.priors

{<gammapy.modeling.models.prior.MultiDimensionalPrior at 0x7efda0389400>: <gammapy.modeling.models.prior.MultiDimensionalPrior at 0x7efda0389400>,
 <gammapy.modeling.models.prior.UniformPriorModel at 0x7efda2c17dc0>: <gammapy.modeling.models.prior.UniformPriorModel at 0x7efda2c17dc0>}

In [26]:
uni = UniformPriorModel(modelparameters = datasets.models.parameters['lon_0'], uni = 0.66)

In [27]:
datasets.priors

{<gammapy.modeling.models.prior.MultiDimensionalPrior at 0x7efda0389400>: <gammapy.modeling.models.prior.MultiDimensionalPrior at 0x7efda0389400>,
 <gammapy.modeling.models.prior.UniformPriorModel at 0x7efda2c17dc0>: <gammapy.modeling.models.prior.UniformPriorModel at 0x7efda2c17dc0>,
 <gammapy.modeling.models.prior.UniformPriorModel at 0x7efd08396850>: <gammapy.modeling.models.prior.UniformPriorModel at 0x7efd08396850>}